In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import sklearn
import sklearn.utils
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
import datetime
from datetime import timedelta
## Load the required module
from sklearn.model_selection import cross_val_score, KFold, cross_validate, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [55]:
pip install datascience

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install prob140

Note: you may need to restart the kernel to use updated packages.


In [57]:
!pip install ipywidgets

In [58]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
from datascience import *
from prob140 import *
%matplotlib inline
plt.style.use('fivethirtyeight')

In [60]:
"""

Dropping nan values and the columns with timestamps and dwell time duration.

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Dataframe with the sequence number, starting latitude and longitude, and ending latitude 
    and longitude for each vin.

"""

df = pd.read_csv("/work/Data/newdataset.csv", index_col= [0,1])
df = df.dropna().drop(columns=['first', 'last', 'dwell time duration (seconds)'])
df

,,sequence,start_lat,start_long,latitude,longitude
,vin,,,,,
2,005b420fd9275f080ad79e1c4330357e,1143,41.267,-81.378,41.410,-81.500
3,005b420fd9275f080ad79e1c4330357e,1145,41.394,-81.413,41.414,-81.504
4,005b420fd9275f080ad79e1c4330357e,1146,41.409,-81.497,41.379,-81.427
6,008beb5671fb49a92dc0ef9de96979bf,555,40.100,-83.166,40.149,-82.904
7,008beb5671fb49a92dc0ef9de96979bf,556,40.149,-82.904,40.100,-83.166
...,...,...,...,...,...,...
48212,ffffe9555b16c2d737f6f2e8616b1e68,644,40.178,-82.997,39.943,-82.827
48213,ffffe9555b16c2d737f6f2e8616b1e68,645,39.942,-82.827,40.178,-82.997
48214,ffffe9555b16c2d737f6f2e8616b1e68,646,40.178,-82.997,40.136,-82.996


In [61]:
"""

Resetting the index and dropping the first column. 

Finding the car with the highest number of trips for the purpose of testing.
    
    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe with the vin, sequence number, starting latitude and longitude, and ending latitude
    for the car with the highest number of trips.
    
"""
df = df.reset_index().drop(columns=["level_0"])
car = df.groupby("vin").agg('count').sort_values(by="start_lat", ascending=False).iloc[0, :]
car_w_most_trips = df[df["vin"] =="f9f14cc8f0891d7158cb72ffb949351b"]
car_w_most_trips

,vin,sequence,start_lat,start_long,latitude,longitude
45843,f9f14cc8f0891d7158cb72ffb949351b,1595,39.906,-82.961,39.936,-82.939
45844,f9f14cc8f0891d7158cb72ffb949351b,1596,39.936,-82.930,39.928,-82.926
45845,f9f14cc8f0891d7158cb72ffb949351b,1598,39.928,-82.925,40.050,-83.123
45846,f9f14cc8f0891d7158cb72ffb949351b,1599,40.050,-83.124,40.068,-83.020
45847,f9f14cc8f0891d7158cb72ffb949351b,1601,40.068,-83.020,40.171,-82.911
...,...,...,...,...,...,...
46140,f9f14cc8f0891d7158cb72ffb949351b,2057,39.979,-82.861,39.944,-82.937
46141,f9f14cc8f0891d7158cb72ffb949351b,2059,39.944,-82.937,39.906,-82.961
46142,f9f14cc8f0891d7158cb72ffb949351b,2060,39.906,-82.961,39.938,-82.793
46143,f9f14cc8f0891d7158cb72ffb949351b,2062,39.938,-82.793,39.938,-82.793


In [62]:
"""

Adding a buffer to allow a  wiggle room of + or - 0.01 from the starting state or location for each car and trip.

Each car can have the same trips to the same place, so we need to find all unique locations for each vin.

Start with an empty dictionary and then add start lat and long locations for one car into it.

This cell was only used to view the output of the dictionary with locations and was later 
added into a function that works for every car.

"""
num = 1
locations = {}
my_buffer = 0.01
for i in np.arange(len(car_w_most_trips)):
    in_chain = False
    lat = car_w_most_trips.iloc[i, 2]
    lon = car_w_most_trips.iloc[i, 3]
    if (i == 0):
        locations[num] = [lat, lon]
        num+=1
        continue
    for val in locations.values():
        lat_1 = val[0]
        lon_1 = val[1]
        if ((abs(lat_1 - lat) < my_buffer) and (abs(lon_1 - lon) < my_buffer)):
            in_chain = True
            break
    if (not in_chain):
        locations[num] = [lat, lon]
        num+=1
locations

{1: [39.906, -82.961],
 2: [39.936, -82.93],
 3: [40.05, -83.124],
 4: [40.068, -83.02],
 5: [40.171, -82.911],
 6: [40.034, -83.033],
 7: [39.938, -82.793],
 8: [39.911, -82.783],
 9: [40.074, -82.853],
 10: [39.957, -82.901],
 11: [39.881, -83.003],
 12: [39.845, -82.823],
 13: [39.917, -82.916],
 14: [39.979, -82.861],
 15: [40.052, -83.021],
 16: [39.948, -82.939],
 17: [39.96, -82.992],
 18: [39.924, -82.832],
 19: [40.032, -83.115],
 20: [39.915, -82.877],
 21: [39.925, -82.816],
 22: [40.059, -82.969],
 23: [40.054, -82.914],
 24: [39.909, -82.824],
 25: [39.923, -82.965],
 26: [39.893, -82.957],
 27: [40.055, -83.067],
 28: [39.996, -83.027],
 29: [40.034, -83.017],
 30: [39.956, -82.869]}

In [63]:
"""

Function that finds all the possible locations that each car might go to next given its latest 
location with a vin and dataframe argument.

Added a buffer to allow a  wiggle room of + or - 0.01 from the starting state or location for 
each car and trip since each car can have the same trips to the same place. Thus, we need to find 
all unique locations for each vin.

Added the location in the first row to begin with and didn't add locations that fell within the buffer zone
since they could've been considered the same. If the next location wasn't in the chain or buffer, 
then we added it to the dictionary as a new location.

Start with an empty dictionary and then add start lat and long locations for one car into it.

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    vin_string = string, vin number
    dataframe = updateed and filtered dataframe from above
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    num = int, number of locations added to the dictionary
    locations = dictionary, contains keys for each state and the corresponding lat and long for each
    my_buffer = float, wiggle room of + or - 0.01
    new_df = dataframe, only for one car (vin_string) and all columns
    start_lat = float, start latitude for each trip for a given car
    start_lon = float, start longitude for each trip for a given car
    lat_1 = float, latitude for coordinates in location dictionary
    lon_1 = float, latitude for coordinates in location dictionary

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Dictionary with all possible locations a given car might go to next.

"""
def vehicle_locations(vin_string, dataframe):
    num = 1
    locations = {}
    my_buffer = 0.01
    new_df = dataframe[dataframe["vin"] == vin_string]
    for i in np.arange(len(new_df)):
        in_chain = False
        start_lat = new_df.iloc[i, 2]
        start_lon = new_df.iloc[i, 3]
        if (i == 0):
            locations[num] = [start_lat, start_lon]
            num+=1
            continue
        for val in locations.values():
            lat_1 = val[0]
            lon_1 = val[1]
            if ((abs(lat_1 - start_lat) < my_buffer) and (abs(lon_1 - start_lon) < my_buffer)):
                in_chain = True
                break
        if (not in_chain):
            locations[num] = [start_lat, start_lon]
            num+=1
    return locations

In [73]:
"""

Make a Markov function for going from one state to another. This was only used to test the transition
matrix output, but we improved it later on in this notebook as make_markov_function().

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    i = int, current state (key for location in locations dictionary)
    j = int, end state (key for location in locations dictionary)
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    i_lat = float, latitude for starting state i
    i_lon = float, longitude for starting state i
    j_lat = float, latitude for every other state j in the dictionary
    j_lon = float, longitude for every other state j in the dictionary
    first = dataframe, filter by start latitudes for state i that fall within the buffer
    second = dataframe, filter by start longitudes for state i that fall within the buffer
    third = dataframe, filter by end latitudes for state j that fall within the buffer
    last = dataframe, filter by end longitudes for state j that fall within the buffer

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Probability for going from one state to another based on locations in the dictionary.

"""
def markov(i, j):
    i_lat = locations.get(i)[0]
    i_lon = locations.get(i)[1]
    j_lat = locations.get(j)[0]
    j_lon = locations.get(j)[1]

    first = car_w_most_trips[abs(car_w_most_trips["start_lat"] - i_lat) <= 0.01]
    second = first[abs(first["start_long"]- i_lon) <= 0.01]
    third = second[abs(second["latitude"] - j_lat) <= 0.01]
    last = third[abs(third["longitude"] - j_lon) <= 0.01]
    return len(last)/len(second)

In [67]:
"""

Checking the markov matrix for the one vehicle with most trips to visualize output.

"""
locations_len = len(vehicle_locations("f9f14cc8f0891d7158cb72ffb949351b", df))
s = np.arange(1, locations_len + 1)

reflecting_walk = MarkovChain.from_transition_function(s, markov)
reflecting_walk

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
1,0.060000,0.600000,0.080000,0.040000,0.000000,0.000000,0.080000,0.020000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.314815,0.111111,0.296296,0.037037,0.000000,0.018519,0.037037,0.018519,0.018519,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.083333,0.000000,0.625000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.041667,0.000000,0.000000,0.000000
4,0.025000,0.025000,0.025000,0.150000,0.025000,0.575000,0.000000,0.000000,0.100000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.025000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.250000,0.000000
6,0.270833,0.062500,0.000000,0.125000,0.020833,0.166667,0.020833,0.000000,0.041667,0.041667,...,0.0,0.083333,0.000000,0.0,0.020833,0.0,0.000000,0.020833,0.041667,0.000000
7,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.200000,0.400000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.125000,0.000000,0.875000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
10,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [68]:
"""

Make a Markov function for all locations and states in the dictionary.

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    df = filtered dataframe created in the first cell of this notebook
    vin = string, vehicle id number
    locations = dictionary with all the next possible locations for a given vehicle
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    car = filtered dataframe for only one car
    i_lat = float, latitude for starting state i
    i_lon = float, longitude for starting state i
    j_lat = float, latitude for every other state j in the dictionary
    j_lon = float, longitude for every other state j in the dictionary
    i_lat_buffer = dataframe, filter by start latitudes for state i that fall within the buffer
    i_lon_buffer = dataframe, filter by start longitudes for state i that fall within the buffer
    j_lat_buffer = dataframe, filter by end latitudes for state j that fall within the buffer
    j_lon_buffer = dataframe, filter by end longitudes for state j that fall within the buffer

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Markov function.

"""
def make_markov_function(df, vin, locations):
    car = df[df["vin"] == vin]
    def markov_function(i, j):
        i_lat = locations.get(i)[0]
        i_lon = locations.get(i)[1]
        j_lat = locations.get(j)[0]
        j_lon = locations.get(j)[1]

        i_lat_buffer = car[abs(car["start_lat"] - i_lat) <= 0.01]
        i_lon_buffer = i_lat_buffer[abs(i_lat_buffer["start_long"]- i_lon) <= 0.01]
        if(len(i_lon_buffer) == 0):
            return 0
        j_lat_buffer = i_lon_buffer[abs(i_lon_buffer["latitude"] - j_lat) <= 0.01]
        j_lon_buffer = j_lat_buffer[abs(j_lat_buffer["longitude"] - j_lon) <= 0.01]
        return len(j_lon_buffer)/len(i_lon_buffer)
    return markov_function

In [69]:
"""
Predicts next location given the vehicle's last recorded location in the dataframe.

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    df = filtered dataframe created in the first cell of this notebook
    vin = string, vehicle id number
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    locations = dictionary using vehicle_locations function from above
    car = filtered dataframe for a given vin
    last_row = most recent trip for one car
    last_lat = float, most recent latitude for one car
    last_lon = float, most recent longitude for one car
    trans_func = transition function with all states and probabilities
    num = list, contains all location keys
    s = array, length of num list
    random_walk = creates random walk for transition function
    state = int, current state the vehicle is in
    i_lat = latitude of given location in dictionary
    i_lon = longitude of given location in dictionary
    probabilities = list, contains the chances that our car moves from current state to any of the other already visited states

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Location with the highest probability that the car will most likely go to next.

"""
def get_next_location(vin, df):
    locations = vehicle_locations(vin, df)
    car = df[df["vin"] == vin]
    last_row = car.tail(1)
    last_lat = float(last_row["latitude"])
    last_lon = float(last_row["longitude"])

    trans_func = make_markov_function(df, vin, locations)
    num = list(locations.keys())[-1]
    s = np.arange(1, num+1)
    random_walk = MarkovChain.from_transition_function(s, trans_func)
    
    state = -1 #starting with a state that doesn't exist
    for key in locations.keys():
        i_lat = locations[key][0]
        i_lon = locations[key][1]
        if ((abs(i_lat - last_lat) <= 0.01) and (abs(i_lon - last_lon) <= 0.01)):
            state = key
            break
    if (state == -1):
        print("This car is in a new location. We cannot predict yet. sorry :(")
        return
    probabilities = [] 
    for i in s:
        probabilities.append(trans_func(state, i))
    max_index = probabilities.index(max(probabilities))
    pred_state = max_index + 1
    return locations[pred_state]

In [70]:
"""
Improved version of the get_next_location() function above.

Predicts all next possible locations given the vehicle's last recorded location in the dataframe.

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    df = filtered dataframe created in the first cell of this notebook
    vin = string, vehicle id number
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    locations = dictionary using vehicle_locations function from above
    car = filtered dataframe for a given vin
    last_row = most recent trip for one car
    last_lat = float, most recent latitude for one car
    last_lon = float, most recent longitude for one car
    trans_func = transition function with all states and probabilities
    num = list, contains all location keys
    s = array, length of num list
    random_walk = creates random walk for transition function
    state = int, current state the vehicle is in
    i_lat = latitude of given location in dictionary
    i_lon = longitude of given location in dictionary
    probabilities = list, contains the chances that our car moves from current state to any of the other already visited states
    existing_lat = float, latitude for existing state in dictionary
    existing_lon = float, longitude for existing state in dictionary
    good_lat = dataframe, filtered by removing latitudes in the buffer zone
    good_lat_n_lon = dataframe, filtered by removing both latitudes and longitudes in the buffer zone
    proportion = int, proportion of possible locations within the overall dataset
    sorted_locs = list, zipped probabilities and locations
    sorted_probs = list, arranging all the locations based on probabilities from sorted_locs in descending order
    df_latitudes = list, latitudes from sorted_locs
    df_longitudes = list, longitudes from sorted_locs
    df_probs = list, all probabilities for each location
    prob_sum = int, sum of probabilities in sorted probability list (only considering sum of 80% for the most likely locations)
    predicted_loc_df = dataframe, all predicted futures locations and chances for each

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Dataframe with all the predicted latitudes and longitudes a given vehicle might go to next with the corresponding probabilities.
    
"""

def get_all_locations(vin, df):
    locations = vehicle_locations(vin, df)
    car = df[df["vin"] == vin]
    last_row = car.tail(1)
    last_lat = float(last_row["latitude"])
    last_lon = float(last_row["longitude"])

    trans_func = make_markov_function(df, vin, locations)
    num = list(locations.keys())[-1]
    s = np.arange(1, num+1)
    walk = MarkovChain.from_transition_function(s, trans_func)
    
    state = -1 #starting with a state that doesn't exist
    for key in locations.keys():
        i_lat = locations[key][0]
        i_lon = locations[key][1]
        if ((abs(i_lat - last_lat) <= 0.01) and (abs(i_lon - last_lon) <= 0.01)):
            state = key
            break
    
    probabilities = []
    if (state == -1):
        for existing_state in locations.keys():
            existing_lat = locations[existing_state][0]
            existing_lon = locations[existing_state][1]
            good_lat = car[abs(car["latitude"] - existing_lat) <= 0.01]
            good_lat_n_lon = good_lat[abs(good_lat["longitude"] - existing_lon) <= 0.01]
            proportion = len(good_lat_n_lon)/len(car)
            probabilities.append(proportion)
    else:
        for i in s:
            probabilities.append(trans_func(state, i))
    
   
    sorted_locs = [x for _, x in sorted(zip(probabilities, locations.values()), reverse=True)]
    sorted_probs = sorted(probabilities, key = lambda x:float(x), reverse=True)
    
    df_latitudes = []
    df_longitudes = []
    df_probs = []
    prob_sum = 0
    for i in range(len(sorted_probs)):
        if sorted_probs[i] > 0 and prob_sum < 0.8:
            df_latitudes.append(sorted_locs[i][0])
            df_longitudes.append(sorted_locs[i][1])
            df_probs.append(sorted_probs[i])
            prob_sum += sorted_probs[i]
            # print("Location: ", sorted_locs[i] , ", Chance: " , sorted_probs[i]*100, "%")
    predicted_loc_df = pd.DataFrame({"pred_lat": df_latitudes, "pred_lon": df_longitudes, "prob": df_probs})
    return predicted_loc_df

In [71]:
"""
Converts all top future locations with corresponding probabilities for each vehicle in the dataset into a dataframe 
and appends each one into a large dataframe.

    Arguments:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    df = filtered dataframe created in the first cell of this notebook
    
    Parameters:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    all_vins = series of strings for all vins in the dataframe
    master_df = dataframe with columns for vin, predicted latitude, predicted longitude, and probabilities
    baby_df = dataframe with future locations and probabilities for one car

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _ 
    Large dataframe for all vehicles in the dataset with top future locations and corresponding probabilities.
    
"""
def master_location_predictions(df):
    all_vins = df["vin"].unique()
    master_df = pd.DataFrame(columns=["vin", "pred_lat", "pred_lon", "prob"])
    for vin in all_vins:
        baby_df = get_all_locations(vin, df)
        baby_df["vin"] = vin
        baby_df = baby_df[["vin", "pred_lat", "pred_lon", "prob"]]
        master_df = master_df.append(baby_df)
    return master_df

In [72]:
"""

View large dataframe from master_location_predictions() function.

"""

master_dataframe = master_location_predictions(df)
master_dataframe.head(10)

,vin,pred_lat,pred_lon,prob
0,005b420fd9275f080ad79e1c4330357e,41.409,-81.497,0.666667
0,008beb5671fb49a92dc0ef9de96979bf,40.149,-82.904,0.750000
1,008beb5671fb49a92dc0ef9de96979bf,40.148,-82.921,0.250000
0,00c85a67f339889c5cb0f5407156c01e,40.087,-82.833,0.444444
1,00c85a67f339889c5cb0f5407156c01e,40.996,-81.997,0.111111
2,00c85a67f339889c5cb0f5407156c01e,40.147,-82.937,0.111111
3,00c85a67f339889c5cb0f5407156c01e,40.132,-83.017,0.111111
4,00c85a67f339889c5cb0f5407156c01e,39.931,-82.834,0.111111
0,00daedebd0d851923f840d67403e58a6,41.492,-81.487,0.250000
0,00f3381dd2f2ab78011c562fa6127df7,39.959,-82.985,0.464286


In [74]:
"""

Convert large dataframe from the output of master_location_predictions() into csv file to add dwell times for each.

"""
master_dataframe.to_csv('/work/Data/locationpredictions.csv')

In [75]:
get_all_locations("01652f6c850a4ed78124a8f06af4391b", df)

,pred_lat,pred_lon,prob
0,39.968,-83.010,0.461538
1,40.020,-83.058,0.115385
2,40.127,-83.124,0.076923
3,40.103,-83.110,0.076923
4,40.098,-83.122,0.076923


In [76]:
get_all_locations("f9f14cc8f0891d7158cb72ffb949351b", df)

,pred_lat,pred_lon,prob
0,39.906,-82.961,0.5
1,39.917,-82.916,0.2
2,39.938,-82.793,0.1
3,39.909,-82.824,0.1


In [77]:
get_next_location("008beb5671fb49a92dc0ef9de96979bf", df)
# should be [40.149, -82.904]

[40.149, -82.904]

In [78]:
get_next_location("01652f6c850a4ed78124a8f06af4391b", df)
# should be [39.968, -83.01]

[39.968, -83.01]

In [79]:
#Test with real dataset
df_new = pd.read_csv("/work/Data/merged_csv_0506.csv")
df_new

,Unnamed: 0,vin,sequence,first,last,start_lat,start_long,latitude,longitude,dwell time duration (seconds),predicted dwell interval greater than 10 hours,prob
0,0,005b420fd9275f080ad79e1c4330357e,1146,1.578953e+09,1.578954e+09,41.409,-81.497,41.379,-81.427,14185.917,0,NaN
1,1,008beb5671fb49a92dc0ef9de96979bf,556,1.578482e+09,1.578484e+09,40.149,-82.904,40.100,-83.166,37768.558,0,NaN
2,2,008beb5671fb49a92dc0ef9de96979bf,559,1.578569e+09,1.578571e+09,40.149,-82.904,40.100,-83.167,37819.974,0,NaN
3,3,008beb5671fb49a92dc0ef9de96979bf,562,1.578697e+09,1.578698e+09,40.149,-82.904,40.058,-82.915,79243.785,0,NaN
4,4,008beb5671fb49a92dc0ef9de96979bf,567,1.578747e+09,1.578747e+09,40.149,-82.904,40.128,-82.932,33563.998,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
29317,29317,ffabe3f9905f857b5f4c64150c4945f0,127,1.580767e+09,1.580768e+09,40.084,-82.840,40.076,-82.837,1915.343,0,NaN
29318,29318,ffabe3f9905f857b5f4c64150c4945f0,21,1.578698e+09,1.578700e+09,40.069,-82.523,40.084,-82.840,2815.011,0,NaN
29319,29319,ffabe3f9905f857b5f4c64150c4945f0,25,1.578766e+09,1.578766e+09,40.061,-82.875,40.079,-82.852,178.002,0,NaN
29320,29320,ffabe3f9905f857b5f4c64150c4945f0,122,1.580693e+09,1.580694e+09,40.061,-82.875,40.084,-82.840,12030.194,0,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=608d4940-9d52-4d53-8e1b-b72eb309e61f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>